In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load scripts from parent path
import sys, os
sys.path.insert(0, os.path.abspath('..'))

## Load Data

In [2]:
#from scripts.processing_copy_Maryam import load_train_data, process_data, add_store_info, add_week_month_info
import scripts.processing_copy_Maryam as scr
train_raw = scr.load_train_data()
train = scr.add_week_month_info(train_raw)
train = scr.process_data(train)
train = scr.add_store_info(train)

train.head()
#X_train, X_test, y_train, y_test = scr.prepare_data(train, test_size = 0.2)

/home/maryam/Github Rep/minicomp-rossman/scripts/processing_copy_Maryam.py:31: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train.loc[:,'week'] = train.loc[:,'Date'].dt.week


,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,week,month,Store_Sales_mean,Store_Customers_mean
0,353.0,2.0,3139.0,1.0,0.0,a,1.0,1,1,4139.474576,1153.783333
1,335.0,2.0,2401.0,1.0,0.0,a,1.0,1,1,12845.896552,2384.271186
2,512.0,2.0,2646.0,1.0,0.0,a,1.0,1,1,3725.649123,888.627119
3,494.0,2.0,3113.0,1.0,0.0,a,1.0,1,1,7079.150000,1010.583333
4,530.0,2.0,2907.0,1.0,0.0,a,1.0,1,1,2260.783333,333.610169


## Prepare train/test data

In [3]:
X = train.copy(deep=True).drop(columns=["Sales"])
y = train.loc[:, "Sales"]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Encoding features

In [6]:
import category_encoders as ce

#ord_enode = ce.OrdinalEncoder(cols=['Foreign_schooled',
 #                                    'Past_Disciplinary_Action','Previous_IntraDepartmental_Movement'])

#base_encode = ce.BaseNEncoder(cols=['State_Of_Origin','Marital_Status'])
one_hot = ce.OneHotEncoder(cols=['StateHoliday'])
target_encode  = ce.TargetEncoder(cols = ['Store'])
ordinal_encode = ce.OrdinalEncoder(cols=['StateHoliday'])

## RandomForestRegressor

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# Drop date column too, makes no sense for random forest
X_train = train.copy(deep=True).drop(columns=["Sales"])
y_train = train.loc[:, "Sales"].values

rfr = RandomForestRegressor()

#build the pipeline
pipe = Pipeline(steps=[('ordinal_encode', ordinal_encode), 
                ('target_encode', target_encode),
               ('model', rfr)])

pipe.fit(X_train, y_train)

Pipeline(steps=[('ordinal_encode',
                 OrdinalEncoder(cols=['StateHoliday'],
                                mapping=[{'col': 'StateHoliday',
                                          'data_type': dtype('O'),
                                          'mapping': a      1
d      2
b      3
c      4
NaN   -2
dtype: int64}])),
                ('target_encode', TargetEncoder(cols=['Store'])),
                ('model', RandomForestRegressor())])

## Model Evaluation

In [11]:
y_pred_train = pipe.predict(X_train)
metric(y_train, y_pred_train)

6.819907688567126

In [12]:
y_pred_test = pipe.predict(X_test)


In [18]:
metric(y_test.values, y_pred_test)

6.8024153726289995

## Simple Models

#### Predict mean

In [ ]:
y_train_withPred =  pd.DataFrame(y_train.copy())
y_test_withPred =  pd.DataFrame(y_test.copy())

y_train_withPred.loc[:,'Sales_pred'] = y_train.mean()
y_test_withPred.loc[:,'Sales_pred']  = y_train.mean()

In [ ]:
metric(y_test_withPred.loc[:,'Sales_pred'].values, y_test.values)